## NovaSAR-1 data products <img align="right" src="../../resources/csiro_easi_logo.png">

#### Index
- [Overview](#Overview)
   - [Tips and tricks](#Tips-and-tricks)
   - [Background on SAR data](#Background-on-SAR-data)
- [Setup (imports, defaults, dask, odc)](#Setup)
- [Example query](#Example-query)
- [Product definition](#Product-definition)
- [Quality layer](#Quality-layer)
- [Create and apply a good quality pixel mask](#Create-and-apply-a-good-quality-pixel-mask)
- [Plot and browse the data](#Plot-and-browse-the-data)

## Overview

This notebook demonstrates how to load and use NovaSAR-1 ARD _gamma-0 backscatter_ data products from the [CSIRO NovaSAR Facility](https://research.csiro.au/cceo/novasar/).

The [NovaSAR-1 data products](https://research.csiro.au/cceo/novasar/about/novasar-1-user-guide/#products) include:
- Level-1 product types Multi-Look Detected (GRD, SCD, SRD) and Single Look Complex (SLC)
- Level-2 analysis ready data (ARD) gamma-0 radiometric terrain corrected backscatter

The **Level-1** data products are stored in swath (line, row) raster grids, with a "fake" WGS-84 bounding box grid for ODC indexing. This means they are not suited to `datacube.load()` directly. Instructions TBC.

The **Level-2 ARD** data products are remapped to WGS-84 grids (various resolutions) as part of the gamma-0 backscatter processing. These data can be opened directly with `datacube.load()`.

## Tips and tricks

#### 1. NovaSAR-1 product names

The ODC currently requires a “dataset” to contain all bands that are defined for its parent product. In practice, each of the NovaSAR-1 product types (GRD, SCD, SRD, SLC, ARD) can have different combinations of available polarization bands across all scene acquisitions. This means we need multiple ODC products; one for each combination of polarizations.

> WIP: relax this constraint in ODC so that we can have "optional" bands defined, which will allow us to aggregate polarization combinations for a single product type.

View the coverage of indexed NovaSAR-1 products and datasets in the [CSIRO EASI Explorer "SAR products group"](https://explorer.csiro.easi-eo.solutions/products#synthetic-aperture-radar-sar-group).

#### 2. Load to target grid or crs/resolution

Different scenes (ODC datasets) in a product may have different spatial resolutions depending on the acquisition mode, so its usually appropriate to provide a target grid. For example, use either

- `datacube.load(..., like=target_geobox)`. See [odc-geo](https://github.com/opendatacube/odc-geo) to help create a target "geobox", or use an existing (ODC-compatible) xarray object
- `datacube.load(..., output_crs=target_crs, resolution=target_res, align=target_res/2)`

For convenience and general applicability, default load parameters are defined for all NovaSAR-1 L2 ARD products as:

- `CRS = "epsg:4326" (WGS-84)`, `Pixel_size = 0.0002 deg` (20 m) and `Align = 0.0001` (10 m). 

This means a `datacube.load()` call without specific output grid parameters (no `like`, `output_crs`, `resolution`) will load data into a _WGS-84, 20 m, aligned to pixel centres_ grid with extents given by the `latitude/longitude/x/y/crs` parameters.

> Native NovaSAR-1 L2 ARD products are aligned to pixel centres (AREA_OR_POINT=Point). You should ensure this is taken into account if mapping pixels to a target grid that is approximately the same resolution as the native scene resolution (otherwise you may be shifting by half-pixel). If mapping to a coarser resolution if may not matter as much.

#### 3. Select or filter scenes before loading

Scenes can be pre-filtered using the scene metadata and the `datacube.find_datasets()` function before datasets are loaded into an xarray object. There are two easy (and similar) ways to achieve this:

1. Use `datacube.find_datasets()` with _product, time, space_ search parameters. This returns a list of dataset items that can be filtered manually, e.g. by scene name or scene metadata. Pass your filtered list to `datacube.load(datasets=my_datasets_list, like=target_geobox)`.
   - This is demonstrated below by parsing likely useful metadata for a list of datasets items into a pandas table, for viewing.
1. Create a metadata predicate function that returns `True` if a dataset's metadata satisifies the conditions specified in your function. This predicate function can be passed to `datacube.load(..., dataset_predicate=my_filter_function`), which is then applied internally in a call to `find_datasets`. See [datacube.load() help](https://opendatacube.readthedocs.io/en/latest/api/indexed-data/generate/datacube.Datacube.load.html) for an example.

#### 4. Units and conversions
The `novasar_l2ard_*` data are given in _Digital numbers_ (DN). DNs can be converted to _decibel (dB)_ or _linear amplitude_, and vice-versa, with the following equations. Practical _Xarray_ examples are given below.

Amplitude to/from dB:
```
dB = 20 * log10(DN) + K
DN = 10^((dB-K)/20)

where K is a calibration factor, which for NovaSAR-1 is -83 dB.
```

Digital numbers to/from Amplitude:
```
amplitude = DN / 14125.3754
DN = amplitude * 14125.3754
```

## Background on SAR data

An excellent introduction and overview to using SAR data is provided in the [CEOS Laymans SAR Interpretation Guide](https://ceos.org/ard/files/Laymans_SAR_Interpretation_Guide_3.0.pdf). This guide has also been converted to a set of set of Jupyter notebooks that you can download from https://github.com/AMA-Labs/cal-notebooks/tree/main/examples/SAR.

The SAR instrument on the NovaSAR-1 satellite operates in the ***S-band at approximately 9.4 cm wavelength***. This means that it can "see" objects of about this size and larger, and smaller objects are relatively transparent. Compared to *Sentinel-1 (C-band, 5.6 cm)*, NovaSAR S-band is less sensitive to tree canopies and has better ground penetration.

> The SAR signal responds to the orientation and scattering from surface features of comparable size or larger than the wavelength.
> - A bright backscatter value typically means the surface was orientated perpendicular to the signal incidence angle and most of the signal was reflected back to the satellite (direct backscatter)
> - A dark backscatter value means most of the signal was reflected away from the satellite (forward scattering) and typically responds to a smooth surface (relative to the wavelength) such as calm water or bare soil
> - Rough surfaces (relative to the wavelength) result in diffuse scattering where some of the signal is returned to the satellite.
> - Complex surfaces may result in volume scattering (scattering within a tree canopy) or double-bounce scattering (perpendicular objects such as buildings and structures)
> - The relative backscatter values of co-polarisation (HH, VV) and cross-polarisation (HV) measurements can provide information on the scattering characteristics of the surface features.

Using NovaSAR-1 backscatter data requires interpretation of the data for different surface features, including as these features change spatially or in time. It may also be necessary to carefully consider the incidence angle of the SAR signal relative to the surface features using the *incidence_angle* band or the satellite direction metadata (descending = north to south; ascending = south to north).

## Set up

#### Imports

In [ ]:
# Common imports and settings
import os, sys, re
from pathlib import Path
from IPython.display import Markdown
import pandas as pd
pd.set_option("display.max_rows", None)
import xarray as xr
import numpy as np

# Datacube
import datacube
from datacube.utils.aws import configure_s3_access
import odc.geo.xr                             # https://github.com/opendatacube/odc-geo
from datacube.utils import masking            # https://github.com/opendatacube/datacube-core/blob/develop/datacube/utils/masking.py
from dea_tools.plotting import display_map    # https://github.com/GeoscienceAustralia/dea-notebooks/tree/develop/Tools

# Dask
import dask
from dask.distributed import Client, LocalCluster

# Basic plots
%matplotlib inline
# import matplotlib.pyplot as plt
# plt.rcParams['figure.figsize'] = [12, 8]

# Holoviews
# https://holoviz.org/tutorial/Composing_Plots.html
# https://holoviews.org/user_guide/Composing_Elements.html
import hvplot.xarray
import holoviews as hv
import panel as pn
from datashader import reductions

In [ ]:
# EASI defaults and convenience functions
# easi-tools is in the repo https://github.com/csiro-easi/easi-notebooks
# These are convenience functions for using these notebooks in EASI; comment-out if not required

repo = Path.home() / 'easi-notebooks'    # Change path as necessary
if str(repo) not in sys.path:
    sys.path.append(str(repo))

from easi_tools import initialize_dask, xarray_object_size  # Useful

#### Dask cluster

Using a local _Dask_ cluster is a good habit to get into. It can simplify loading and processing of data in many cases, and it provides a dashboard that shows the loading/processing progress.

To learn more about _Dask_ see the set of [dask notebooks](https://github.com/csiro-easi/easi-notebooks/tree/main/html#dask-tutorials).

In [ ]:
# Local cluster - good for small exploratory and testing work
cluster, client = initialize_dask(workers=8)
display(client)

# Or use Dask Gateway - this may take a few minutes
# cluster, client = initialize_dask(use_gateway=True, workers=4)
# display(client)

#### ODC database

Connect to the ODC database. Configure the environment and low-level tools to read from AWS buckets.

In [ ]:
dc = datacube.Datacube()

# Access AWS "requester-pays" buckets
# This is necessary for reading data from most third-party AWS S3 buckets such as for Landsat and Sentinel-2
configure_s3_access(aws_unsigned=False, requester_pays=True, client=client);

## Example query

Change any of the parameters in the `query` object below to adjust the location, time, projection, or spatial resolution of the returned datasets.

Use the Explorer interface to check the temporal and spatial coverage for each product.

In [ ]:
# Set your own latitude / longitude

# Menindee Lakes, Australia
latitude_range = (-32.1, -33.6)
longitude_range = (141.5, 143)

# Great Western Woodlands, Australia
# latitude_range = (-33, -32.6)
# longitude_range = (120.5, 121)

time_range = None   # All available times

available_ard_products = [
    'novasar_l2ard_hh',         # Moderate coverage, https://explorer.csiro.easi-eo.solutions/products/novasar_l2ard_hh
    'novasar_l2ard_hh_hv',      # High coverage, https://explorer.csiro.easi-eo.solutions/products/novasar_l2ard_hh_hv
    'novasar_l2ard_hv',         # Low coverage, https://explorer.csiro.easi-eo.solutions/products/novasar_l2ard_hv
    'novasar_l2ard_vv',         # Moderate coverage, https://explorer.csiro.easi-eo.solutions/products/novasar_l2ard_vv
    'novasar_l2ard_vv_hh',      # Moderate coverage, https://explorer.csiro.easi-eo.solutions/products/novasar_l2ard_vv_hh
    'novasar_l2ard_vv_hh_hv',   # High coverage, https://explorer.csiro.easi-eo.solutions/products/novasar_l2ard_vv_hh_hv
]

# Select a product
product_name = 'novasar_l2ard_hh_hv'
polarizations = re.findall('_([vh]+)', product_name)  # The set of polarization bands for the selected product

query = {
    'product': product_name,       # Product name
    'measurements': polarizations + ['angle', 'mask', 'scatteringarea', 'gammatosigmaratio'],  # All bands for testing
    'x': longitude_range,     # "x" axis bounds
    'y': latitude_range,      # "y" axis bounds
    'time': time_range,       # Any parsable date strings
}

# Convenience function to display the selected area of interest
display_map(longitude_range, latitude_range)

## Review scene metadata for the query

Run `datacube.find_datasets()` for the query and summarise resulting scene information in a pandas table

In [ ]:
datasets = dc.find_datasets(**query)

# Examples
# datasets[0].metadata_doc --> dict of all metadata
# datasets[0].measurements --> dict of measurements
# datasets[0].grids --> dict of grids (shape and transform) defined for measurements
# datasets[0].crs --> CRS object
# datasets[0].extent --> geometry object
# datasets[0].time --> start and end time objects
# datasets[0].properties --> dict of properties

# Select your own fields
fields = {
    'name': None,
    'time': None,
    'gsd': None,
    'polarizations': None,
    'pass_direction': None,
    'antenna_pointing': None,
    'platform_heading': None,
    'incident_angle_near_range': None,
    'incident_angle_far_range': None,
    'source_product_level': None,
    'operational_mode_name': None,
    'filter_applied': None,
    'noise_removal_applied': None,
}

# Parse into a pandas DataFrame
df = []
for ds in datasets:
    ff = fields.copy()
    # Non-properties fields
    ff['name'] = ds.metadata.label
    ff['time'] = ds.time[0]  # time->range(start, end) or center_time->(end-start)/2
    # Properties fields
    for xx in fields:
        if xx in ('name', 'time'):
            continue
        pre = 'eo' if xx == 'gsd' else 'novasar'
        ff[xx] = ds.properties[f"{pre}:{xx}"]
    df.append(ff)
df = pd.DataFrame(df)
display(df.sort_values('time') if not df.empty else Markdown("**Dataframe is empty**. Choose different product, space, time parameters"))

## Filter scenes based on metadata only

This step is optional and configurable. Here we show a selection of simple filters:

- Select descending passes only
- Select the N most recent scenes

In [ ]:
selected_datasets = datasets  # Start with the full list

# Select descending passes only
selected_datasets = [
    xx for xx in selected_datasets if xx.properties["novasar:pass_direction"] == "DESCENDING"
]

# Select the N most recent scenes
selected_datasets = sorted(selected_datasets, key=lambda ds: ds.time[0])[-10:]

display(Markdown(f"**Number of selected scenes**: {len(selected_datasets)}"))
selected_datasets

## Load data into a virtual dask array

In [ ]:
# Target xarray parameters
# - Target GeoBox or output CRS and resolution
# - Usually we group input scenes on the same day to a single time layer (groupby)
# - Select a reasonable Dask chunk size. This should be adjusted depending on the spatial extents and target grid parameters you choose

# Default target grid is WGS-84, 20 m pixels for the given spatial extents
# - Create or reuse a GeoBox for precise mapping of input pixels to a target grid
# - Or provide output_crs and resolution, which will create best-fit GeoBox

# Example similar Geobox constructor
# geobox = odc.geo.geobox.GeoBox.from_bbox(
#     [query['x'][0], query['y'][1], query['x'][1], query['y'][0]],
#     crs='epsg:4326', resolution=0.0002, anchor=0.0001
# )

load_params = {
    'datasets': selected_datasets,
    'group_by': 'solar_day',                        # Scene grouping
    'dask_chunks': {'latitude':2048, 'longitude':2048},      # Dask chunks
}

# Load data
data = dc.load(**(query | load_params))

display(xarray_object_size(data))
display(data)

## Conversion and helper functions

In [ ]:
# These functions use numpy, which should be satisfactory for most notebooks.
# Calculations for larger or more complex arrays may require Xarray's "ufunc" capability.
# https://docs.xarray.dev/en/stable/examples/apply_ufunc_vectorize_1d.html
#
# Apply numpy.log10 to the DataArray
# log10_data = xr.apply_ufunc(np.log10, data)

def dn_to_decibel(da: 'xr.DataArray', K=0):
    """Return an array converted to dB values"""
    xx = da.where(da > 0, np.nan)  # Set values <= 0 to NaN
    xx = 20*np.log10(xx) + K
    xx.attrs.update({"units": "dB"})
    return xx

def decibel_to_dn(da: 'xr.DataArray', K=0):
    """Return an array converted to digital number values"""
    xx = np.power(10, (da-K)/20.0)
    xx.attrs.update({"units": "DN"})
    return xx

def dn_to_amplitude(da: 'xr.DataArray'):
    """Return an array converted to linear amplitude values"""
    scale_factor = da.attrs['scale_factor'] if 'scale_factor' in da.attrs else 1
    # print(f"DN to amplitude scale_factor: 1/{1/scale_factor} = {scale_factor}")
    xx = da.where(da > 0, np.nan)  # Set values <= 0 to NaN
    xx = xx * scale_factor
    xx.attrs.update({"units": "amplitude"})
    return xx

def select_valid_time_layers(ds: 'xarray', percent: float = 5):
    """Select time layers that have at least a given percentage of valid data (e.g., >=5%)

    Example usage:
      selected = select_valid_time_layers(ds, percent=5)
      filtered == ds.sel(time=selected)
    """
    spatial_dims = ds.odc.spatial_dims
    nelements = ds.sizes[spatial_dims[0]] * ds.sizes[spatial_dims[1]]
    if ds.dtype =='bool':
        return ds.sum(dim=spatial_dims).values / nelements >= (percent/100.0)
    return ds.count(dim=spatial_dims).values / nelements >= (percent/100.0)

# Examples to check that the intensity to/from dB functions work as expected
# xx = data.vv.isel(time=0,latitude=np.arange(0, 5),longitude=np.arange(0, 5))
# xx[0] = 0       # manually change some values
# xx[1] = -0.001  # manually change some values
# display("digital numbers:", xx.values)
# yy = dn_to_decibel(xx, K=-83)
# display("decibels:", yy.values)
# zz = decibel_to_dn(yy, K=-83)
# display("digital numbers:", zz.values)
# aa = dn_to_amplitude(xx)
# display("amplitude:", aa.values)

In [ ]:
# hvPlot convenience functions
def make_image(ds: 'xarray', frame_height=300, **kwargs):
    """Return a Holoviews DynamicMap (image) object that can be displayed or combined"""
    spatial_dims = ds.odc.spatial_dims
    defaults = dict(
        cmap="Greys_r",
        y = spatial_dims[0], x = spatial_dims[1],
        groupby = 'time',
        rasterize = True,
        geo = True,
        robust = True,
        frame_height = frame_height,
        clabel = ds.attrs.get('units', None),
    )
    defaults.update(**kwargs)
    return ds.hvplot.image(**defaults)

def rgb_image(ds: 'xarray', frame_height=300, **kwargs):
    """Return a Holoviews DynamicMap (RBG image) object that can be displayed or combined"""
    spatial_dims = ds.odc.spatial_dims
    defaults = dict(
        bands='band',
        y = spatial_dims[0], x = spatial_dims[1],
        groupby = 'time',
        rasterize = True,
        geo = True,
        robust = True,
        frame_height = frame_height,
    )
    defaults.update(**kwargs)
    return ds.hvplot.rgb(**defaults)

def mask_image(ds: 'xarray', frame_height=300, **kwargs):
    """Return a Holoviews DynamicMap (mask image) object that can be displayed or combined"""
    # NovaSAR ARD mask
    color_def = [
        (0,  '#ff0004', 'InvalidData'),   # red
        (1,  '#eeeeee', 'ValidData'),   # light grey
        (5, '#ff52ff', 'Layover'),   # cyan
        (17,  '#774c0b', 'Shadow'),   # brown
        (18, 'black', 'upper-limit'),
    ]
    cvals = [x[0] for x in color_def]   # values, including upper-limit
    cmap = [x[1] for x in color_def[0:-1]]   # colors, excluding upper-limit
    cticks = [(x[0], f"[{x[0]}] {x[2]}") for x in color_def[0:-1]]  # labels, excluding upper-limit

    # Image options
    defaults = {
        'aggregator': reductions.mode(),
        'cmap': cmap,
        'clim': (cvals[0], cvals[-1]),
        'colorbar': True,
        'frame_height': frame_height,
    }
    # Colorbar options for categories
    extra_opts = {
        'color_levels': cvals,
        'cticks': cticks,
    }
    defaults.update(**kwargs)
    return make_image(ds, **defaults).options(hv.opts.Image(**extra_opts))  #.hist(bins=bin_edges)

## Filter scenes based on valid data

This step is optional and configurable. Here we show another simple filter:

- Exclude time layers with less than XX% valid data

In [ ]:
# Exclude time layers with less than XX% valid data

data['mask'] = data.mask.persist()
valid_data_mask = masking.valid_data_mask(data.mask)  # mask != mask.nodata -> bool
selected = select_valid_time_layers(valid_data_mask, 20)  # Exclude time layers with less than 20% valid data
data = data.sel(time=selected)

display(xarray_object_size(data))
display(data)

## Add dB and amplitude values to the dataset

In [ ]:
# Convenience loop for available polarization bands

for pp in polarizations:
    data[f"{pp}_db"] = dn_to_decibel(data[pp], K=-83).astype('float32')
    data[f"{pp}_amp"] = dn_to_amplitude(data[pp]).astype('float32')

display(xarray_object_size(data))
display(data)

## Persist the data into the dask workers

This will begin the data loading and calculations in the background.

View progress in the dask dashboard (link given above where the dask cluster is defined).

In [ ]:
# Refine for selected variables if not plotting/analysing everything
data = data.persist()

## Plot the data

- Stronger co-polarization (VV) indicates direct backscatter while stronger cross-polarization (VH) may indicate a complex surface or volume scattering.
- Amplitude data are linear-scaled so can tend to disciminate across a range of backscatter returns.
- Decibel data are log-scaled so can tend to discriminate high and low backscatter returns.

> Note the different data ranges for plotting (`clim`) between `vv`, `vh`, _amplitude_ and _dB_.

In [ ]:
# VV, HH and HV (amplitude and dB) and Angle hvPlots

clim = {
    'vv_amp': (0, 0.5),
    'hh_amp': (0, 0.5),
    'hv_amp': (0, 0.25),
    'vv_db': (-20, -5),
    'hh_db': (-20, -5),
    'hv_db': (-35, -10),
}

bands_plot_list = []
for units in ('amplitude', 'dB'):
    for pp in polarizations:
        varname = f"{pp}_{units[0:3].lower()}"
        title = f"{pp.upper()} ({units})"
        bands_plot_list.append(
            make_image(data[varname], title=title, clim=clim[varname])
        )

# Which is essentially doing this for all polarizations
# vv_plot = make_image(data.vv_amp, title='VV (amplitude)', clim=(0, 0.5))
# vv_db_plot = make_image(data.vv_db, title='VV (dB)', clim=(-20, -5))

# # Add plots for the non-polarization bands
bands_plot_list.append(make_image(data.angle, title='Incidence angle'))
bands_plot_list.append(make_image(data.scatteringarea, title='Scattering area'))
bands_plot_list.append(make_image(data.gammatosigmaratio, title='Gamma to sigma ratio'))
bands_plot_list.append(mask_image(data.mask, title='Mask'))

In [ ]:
# Arrange plots with linked axes and time slider. Adjust browser window width if required.

num_plot_cols = 2 if len(polarizations) <= 2 else len(polarizations)
layout = pn.panel(
    hv.Layout(bands_plot_list).cols(num_plot_cols),
    widget_location='top',
)

print(layout)  # Helpful to see how the hvplot is constructed
display(layout)

## Plot histograms of the dB data

A histogram can help separate water from land features. Here we show a histogram for the _dB_ channels for all time layers.
- If the histogram shows two clear peaks then a value between the peaks could be used as a water / land threshold
- If not then try selected time layers, a different area of interest, or other channels or combinations.

In [ ]:
# vals, bins, hist_plot = data.hv_db.plot.hist(bins=np.arange(-30, 0, 1), color='red')  # Matplotlib

hist_plot_list = []
for pp in polarizations:
    varname = f"{pp}_db"
    title = f"{pp.upper()} (dB), combined times"
    hist_plot_list.append(
        data[varname].hvplot.hist(bins=np.arange(-30, 0, 1), color='red', title=title, height=300)
    )

layout = hv.Layout(hist_plot_list).cols(len(polarizations))

print(layout)  # Helpful to see how the hvplot is constructed
display(layout)

## Make an RGB image

A common strategy to create an RGB colour composite image for SAR data from two channels is to use the ratio of the channels to represent the third colour.

Here we choose:

- For a tri-pol (3 polarization bands) product we plot each polarization as red, green and blue.
- For a dual-pol (2 polarization bands) product we plot each polarization as red and green and the ratio as blue.
- For a single-pol (1 polarization band) product we can not make a useful RGB image

Recall that:
- VV or HH ... direct scattering
- VH or HV ... complex scattering
- ratio ... relatively more of one than the other

In [ ]:
# Select bands combinations for RGB
if len(polarizations) == 3:
    rgb_bands = polarizations
elif len(polarizations) == 2:
    combo = "_".join(polarizations)
    data[f"{combo}"] = data[polarizations[0]] / data[polarizations[1]]
    rgb_bands = polarizations + [combo]
else:
    display(Markdown("**Can not make a useful RGB image from one polarization band**. Choose a dual- or tri-pol product"))
    rgb_bands = []

In [ ]:
if rgb_bands:
    # Scale RGB bands by their median so they have a similar range for visualization
    spatial_dims = data.odc.spatial_dims
    for bb in rgb_bands:
        data[f"{bb}_scaled"] = (data[bb] / data[bb].median(dim=spatial_dims))
    rgb_bands = [f"{bb}_scaled" for bb in rgb_bands]

    # odc-geo function
    rgb_data = data.odc.to_rgba(bands=rgb_bands, vmin=0, vmax=2).persist()

    # As subplots
    # rgb_plot = rgb_image(
    #     rgb_data,
    # ).layout().cols(4)

    # As movie. Select "loop" and use "-" button to adjust the speed to allow for rendering. After a few cycles the images should play reasonably well.
    rgb_plot = rgb_image(
        rgb_data,
        precompute = True,
        widget_type='scrubber', widget_location='bottom',
        frame_height = 500,
    )

In [ ]:
if rgb_bands:
    print(rgb_plot)  # Helpful to see how the hvplot is constructed
    display(rgb_plot)

## Export to Geotiffs

Recall that to write a dask dataset to a file requires the dataset to be `.compute()`ed. This may result in a large memory increase on your JupyterLab node if the area of interest is large enough, which in turn may kill the kernel. If so then skip this step, choose a smaller area or find a different way to export data.

In [ ]:
# Make a directory to save outputs to
target = Path.home() / 'output'
if not target.exists(): target.mkdir()

def write_band(ds, varname):
    """Write the variable name of the xarray dataset to a Geotiff file for each time layer"""
    for i in range(len(ds.time)):
        date = ds[varname].isel(time=i).time.dt.strftime('%Y%m%d').data
        fname = f'{target}/example_novasar-1_{varname}_{date}.tif'
        single = ds[varname].isel(time=i).compute()
        single.odc.write_cog(
            fname=fname,
            overwrite=True,
        )
        print(f'Wrote: {fname}')

for pp in polarizations:
    write_band(data, pp)